In [1]:
from keras.datasets import *
from keras.utils import *

boston_housing = boston_housing
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((404, 13), (404, 51), (102, 13), (102, 51))

In [2]:
from keras import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from scikeras.wrappers import *
from sklearn.model_selection import *
from sklearn.metrics import *

score_acc = make_scorer(accuracy_score)

def nn_cl_bo(neurons, activation, optimizer, learning_rate, batch_size, epochs):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    optimizerD = {
        'SGD': SGD(learning_rate=learning_rate),
        'Adam': Adam(learning_rate=learning_rate),
        'RMSprop': RMSprop(learning_rate=learning_rate),
        'Adadelta': Adadelta(learning_rate=learning_rate),
        'Adagrad': Adagrad(learning_rate=learning_rate),
        'Adamax': Adamax(learning_rate=learning_rate),
        'Nadam': Nadam(learning_rate=learning_rate),
        'Ftrl': Ftrl(learning_rate=learning_rate) 
    }
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        opt = Adam(learning_rate=learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = KFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks': [es]}, error_score='raise').mean()
    return score

In [5]:
from keras_tuner import BayesianOptimization

params_nn = {
    'neurons': (10, 100),
    'activation': (0, 7),
    'optimizer': (0, 7),
    'learning_rate': (0.01, 1),
    'batch_size': (200, 1000),
    'epochs': (20, 100)
}

nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(num_initial_points=25, n_iter=4)

TypeError: `objective` not understood, expected str or `Objective` object, found: {'neurons': (10, 100), 'activation': (0, 7), 'optimizer': (0, 7), 'learning_rate': (0.01, 1), 'batch_size': (200, 1000), 'epochs': (20, 100)}